In [1]:
import pandas as pd
import geopandas as gpd

In [10]:
regiones = gpd.read_file('../../Regiones/Regional.shp')
regiones = regiones.to_crs(("EPSG:4326"))

In [11]:
regiones.query('codregion==12')

,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,Region,geometry
3,1087,15,12,133053.142345,3.581316e+11,9.049830e+07,Región de Magallanes y Antártica Chilena,"MULTIPOLYGON (((-67.32026 -55.92015, -67.31971..."


In [12]:
regiones.query('codregion==12').geometry.bounds

,minx,miny,maxx,maxy
3,-75.72192,-55.980002,-66.418214,-48.595635


In [17]:
gpd.GeoSeries(square_geom).to_crs("EPSG:32733")

ValueError: Cannot transform naive geometries.  Please set a crs on the object first.

,geometry
0,"POLYGON ((-3537934.060 -52286.471, -3480452.59..."


In [26]:
import geopandas as gpd
from shapely.geometry import Polygon, Point
import pyproj
import numpy as np

# Define the square geometry in lat/lon
min_lon, max_lon = -75.72192, -66.418214
min_lat, max_lat = -55.980002, -48.595635
square_geom = Polygon([(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)])

# Define the UTM zone to use
lat_0 = (min_lat + max_lat) / 2
lon_0 = (min_lon + max_lon) / 2

# Project the square geometry to UTM
square_utm = (gpd.GeoDataFrame(geometry=gpd.GeoSeries(square_geom), crs="EPSG:4326")).to_crs("EPSG:32733")

In [27]:
# Calculate the number of points needed in the x and y direction
x_span = square_utm.geometry.iloc[0].bounds[2] - square_utm.geometry.iloc[0].bounds[0]
y_span = square_utm.geometry.iloc[0].bounds[3] - square_utm.geometry.iloc[0].bounds[1]
n_points_x = int(x_span / 1000)
n_points_y = int(y_span / 1000)

In [34]:
np.linspace(0,10, 21)

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
        5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ])

In [35]:
x_coords = [square_utm.geometry.iloc[0].bounds[0] + k*1000 for k in range(n_points_x)]

In [36]:
x_coords

[-4576761.9022821,
 -4575761.9022821,
 -4574761.9022821,
 -4573761.9022821,
 -4572761.9022821,
 -4571761.9022821,
 -4570761.9022821,
 -4569761.9022821,
 -4568761.9022821,
 -4567761.9022821,
 -4566761.9022821,
 -4565761.9022821,
 -4564761.9022821,
 -4563761.9022821,
 -4562761.9022821,
 -4561761.9022821,
 -4560761.9022821,
 -4559761.9022821,
 -4558761.9022821,
 -4557761.9022821,
 -4556761.9022821,
 -4555761.9022821,
 -4554761.9022821,
 -4553761.9022821,
 -4552761.9022821,
 -4551761.9022821,
 -4550761.9022821,
 -4549761.9022821,
 -4548761.9022821,
 -4547761.9022821,
 -4546761.9022821,
 -4545761.9022821,
 -4544761.9022821,
 -4543761.9022821,
 -4542761.9022821,
 -4541761.9022821,
 -4540761.9022821,
 -4539761.9022821,
 -4538761.9022821,
 -4537761.9022821,
 -4536761.9022821,
 -4535761.9022821,
 -4534761.9022821,
 -4533761.9022821,
 -4532761.9022821,
 -4531761.9022821,
 -4530761.9022821,
 -4529761.9022821,
 -4528761.9022821,
 -4527761.9022821,
 -4526761.9022821,
 -4525761.9022821,
 -4524761.90

In [14]:
# Generate a mesh of points within the square geometry
x_coords = np.linspace(square_utm.geometry.iloc[0].bounds[0], square_utm.geometry.iloc[0].bounds[2], n_points_x)
y_coords = np.linspace(square_utm.geometry.iloc[0].bounds[1], square_utm.geometry.iloc[0].bounds[3], n_points_y)
mesh_points = np.array(np.meshgrid(x_coords, y_coords)).T.reshape(-1, 2)
mesh_geom_utm = [Point(xy) for xy in mesh_points]
mesh_gdf_utm = gpd.GeoDataFrame(geometry=mesh_geom_utm, crs=square_utm.crs)

# Project the mesh points back to lat/lon
mesh_gdf_latlon = gpd.GeoDataFrame(geometry=gpd.GeoSeries(mesh_gdf_utm.geometry).to_crs('EPSG:4326'), crs='EPSG:4326')

CRSError: Invalid projection: +proj=utm +zone=19S +type=crs: (Internal Proj Error: proj_create: Error 1027 (Invalid value for an argument): utm: Invalid value for zone)